In [ ]:
import keras
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K
import matplotlib.pyplot as plt
import cv2
import os
from keras.layers.advanced_activations import LeakyReLU


input_img = Input(shape=(28, 28, 1))  # adapt this if using `channels_first` image data format

#initial_mode = keras.initializers.glorot_normal(seed=None)


#activation_method = 'relu'
#activation_method = LeakyReLU(alpha=0.5)
#activation_method=tf.keras.layers.LeakyReLU(alpha=0.1)

#kernel_initializer=initial_mode

for i in range(1,2):
    ## Build the Convolution autoencoders model
    
    x = Conv2D(16, (3, 3), padding='same')(input_img)# 28*28*16    
    x = LeakyReLU(alpha=0.5)(x)
    x = MaxPooling2D((2, 2), padding='same')(x)  # 14*14*16
    
    x = Conv2D(8, (3, 3), padding='same')(x) # 14*14*8
    x = LeakyReLU(alpha=0.5)(x)
    x = MaxPooling2D((2, 2), padding='same')(x) # 7*7*8
    
    x = Conv2D(8, (3, 3), padding='same')(x) # 7*7*8
    x = LeakyReLU(alpha=0.5)(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x) # 4*4*8

    # This part is full connect layer
    # at this point the representation is (4, 4, 8) i.e. 128-dimensional

    x = Conv2D(8, (3, 3), padding='same')(encoded) # 4*4*8
    x = LeakyReLU(alpha=0.5)(x)

    x = UpSampling2D((2, 2))(x) # 8*8*8
    x = Conv2D(8, (3, 3), padding='same')(x) # 8*8*8
    x = LeakyReLU(alpha=0.5)(x)

    x = UpSampling2D((2, 2))(x) # 16*16*8
    x = Conv2D(16, (3, 3))(x) # 14*14*16
    x = LeakyReLU(alpha=0.5)(x)

    x = UpSampling2D((2, 2))(x) # 28*28*16
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x) # 28*28*1
    


    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer='nadam', loss='binary_crossentropy')

    from keras.datasets import mnist
    import numpy as np

    (x_train, _), (x_test, _) = mnist.load_data()

    x_train = x_train.astype('float32') / 255.
    x_test = x_test.astype('float32') / 255.
    x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))  # adapt this if using `channels_first` image data format
    x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))  # adapt this if using `channels_first` image data format

    from keras.callbacks import TensorBoard

    

    
    class CustomSaver(keras.callbacks.Callback):
        def on_epoch_begin(self, epoch, logs={}):            
            loop = epoch + 1
            
            if epoch ==0:
                print('first epoch')

            
            elif loop%5 == 0:  # or save after some epoch, each k-th epoch etc.
                print('Save the image')
                decoded_imgs = autoencoder.predict(x_test)
                img = decoded_imgs[0]
                img = img.reshape(28,28)
                
                # save the img
                current_dir = os.getcwd()
                plt.imshow(img,'gray')
                plt.savefig(current_dir+'/Decoded_img/Decoded_image_epoch'+str(loop)+'.png')
           
            
                

    saver = CustomSaver()
    current_dir = os.getcwd()

    result = autoencoder.fit(x_train, x_train,
                                        epochs=50,
                                        batch_size=128,
                                        shuffle=True,
                                        validation_data=(x_test, x_test),
                                        callbacks=[saver,TensorBoard(log_dir=current_dir+'/tensorboard_data/result_a_g')])
    
    


#print(autoencoder.summary())




#decoded_images = autoencoder.predict(x_test)

In [ ]:
decoded_imgs = autoencoder.predict(x_test)
print(autoencoder.summary())

In [ ]:
b = x_test[0]
b = b.reshape(28,28)
plt.imshow(b,'gray')


In [ ]:
decoded_imgs.shape
a = decoded_imgs[0]
a = a.reshape(28,28)
plt.imshow(a,'gray')


In [ ]:
autoencoder.save('my_model.h5')

In [ ]:
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import cv2
import os,sys

current_dir = os.getcwd()
new_model = tf.keras.models.load_model(current_dir+'/model/result/test_a_g.h5')


In [ ]:
print(new_model.summary())

In [ ]:
# Define the test model

model_name = 'test_a_g'

#test_img.shape


In [ ]:
#Do test 1

current_dir = os.getcwd()
test_img = cv2.imread('test1.png',cv2.IMREAD_GRAYSCALE)

test_img = abs(255-test_img)
test_img = test_img/255

#change the input shape
new_img = cv2.resize(test_img,(28, 28))
new_img.shape
plt.imshow(new_img,cmap='gray')
#plt.savefig(model_name+'_number1.png')

new_img1 = new_img.reshape(1,28,28,1) 

In [ ]:
pred = new_model.predict(new_img1)

output = pred.reshape(28,28) 

plt.imshow(output,cmap='gray')
plt.savefig(model_name+'_result_number1.png')

In [ ]:
#Do test 2

current_dir = os.getcwd()
test_img = cv2.imread('test2.jpeg',cv2.IMREAD_GRAYSCALE)

ret, test_img = cv2.threshold(test_img, 130, 255, cv2.THRESH_BINARY)

test_img = abs(255-test_img)
test_img = test_img/255

#change the input shape
new_img = cv2.resize(test_img,(28, 28))
#new_img.shape
plt.imshow(new_img,cmap='gray')
#plt.savefig('test_src.png')

new_img2 = new_img.reshape(1,28,28,1) 

In [ ]:
pred = new_model.predict(new_img2)

output = pred.reshape(28,28) 

plt.imshow(output,cmap='gray')
plt.savefig(model_name+'_result_number7.png')